## The task consist to classify the semantic of tweets. To deal with a deep understanding of natural language I'll try directly to handle the task with the transformer achitecture.


### I'll use the HF library

In [1]:
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

%pip install \
    transformers \
    datasets \
    evaluate --quiet

%pip install accelerate -U --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from transformers import  AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification
import torch
import time
import evaluate
import pandas as pd
import numpy as np

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


### Check if it's running on the GPU

In [4]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
Device name: Tesla T4


### Let's start by importing the train dataset and see the first few lines to get a better understanging of the structure.

In [5]:
train_path = '/kaggle/input/nlp-getting-started/train.csv'

df = pd.read_csv(train_path)

df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
print(df.text[554])
print(df.text[67])
print(df.text[34])
print(df.text[45])
print(df.text[340])

#Tb #throwback ??

??~ You want a battle? Here's a War! ~ ?? https://t.co/B0ZJWgmaIW
'I can't have kids cuz I got in a bicycle accident &amp; split my testicles. it's impossible for me to have kids' MICHAEL YOU ARE THE FATHER
Crying out for more! Set me ablaze
I gained 3 followers in the last week. You? Know your stats and grow with http://t.co/TIyUliF5c6
Lee does comedy: ÛÏ@LeeJasper: Working class Tories prepare for your Armageddon. #InterestRateRiseÛ


### Now I can see that the most important feature that I need is "text". The features "keyword" and "location" have many missing values already in the first few lines so let's check deply if they could be usuable or not.

In [7]:
print("Missing values in keyword feature: ", df.shape[0] - df.keyword.isna().value_counts()[0])
print("Missing values in location feature: ", df.shape[0] - df.location.isna().value_counts()[0])
print("Missing values in text feature: ", df.shape[0] - df.text.isna().value_counts()[0])
print("Missing values in target feature: ", df.shape[0] - df.target.isna().value_counts()[0])

Missing values in keyword feature:  61
Missing values in location feature:  2533
Missing values in text feature:  0
Missing values in target feature:  0


### I see a lot of information is missing about the location, so I'll not focus on that one. Instead let's see how is the keyword feature

In [8]:
df.keyword.value_counts()

fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

### I think it might be usefull to add this feature too in the model, I tried also without but the accuracy on a light dataset was lower

### After I made some tests, I wrote a two functions to handle the data cleaning, by removing part of the text that cannot be handle well by the tokenizer

In [9]:
import re

def text_clean(text):

    ### Light
    #text = text.lower() # lowercase everything
    text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
    text = re.sub(r'https*\S+', ' ', text) # remove links
    text = re.sub(r'http*\S+', ' ', text)
    # cleaning up text
    #text = re.sub(r'%20', '', text)
    text = text.replace("%20", " " )
    text = re.sub(r'\'\w+', '', text)
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()


    return text

In [10]:
def prepare_promt(df_to_clean):
    df_to_clean.keyword.fillna("none", inplace=True)
    df_to_clean["text"] = "" + df_to_clean["text"] + " [SEP] keyword: " + df_to_clean["keyword"]
    df_to_clean['text'] = df_to_clean.text.apply(text_clean)
    return df_to_clean

### Now call the function to clean the text

In [11]:
df = prepare_promt(df)

In [12]:
print(df.text[554])
print(df.text[67])
print(df.text[34])
print(df.text[45])
print(df.text[340])

#Tb #throwback ?? ??~ You want a battle? Here a War!?? [SEP] keyword: battle
can have kids cuz I got in a bicycle accident & split my testicles. it impossible for me to have kids' MICHAEL YOU ARE THE FATHER [SEP] keyword: accident
Crying out for more! Set me ablaze [SEP] keyword: ablaze
I gained followers in the last week. You? Know your stats and grow with [SEP] keyword: ablaze
Lee does comedy: Working class Tories prepare for your Armageddon. #InterestRateRise [SEP] keyword: armageddon


### It seems fine. Now it's time to create a HF dataset since to utilize LLM models with HF it's required

In [13]:
from datasets import Dataset

df = df.drop(columns=['id','keyword','location'])
df.rename(columns={'target': 'label'}, inplace=True) # the transformer model require that the Y is called label


dataset = Dataset.from_pandas(df) # load Dataset from Pandas DataFrame

dataset = dataset.train_test_split(test_size=0.2) # I split the dataset in two so that I can use an unseen part as evaluation data

print(dataset)

print(dataset["train"][100])
print(dataset["test"][10])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1523
    })
})
{'text': 'Shadowflame and the Wraith: Bombed via [SEP] keyword: bombed', 'label': 1}
{'text': 'New cocktail on the list! El Diablo Mas Verde: mezcal yellow chartreuse honey cucumber hellfire bitters.... [SEP] keyword: hellfire', 'label': 0}


### To finetune the llm I need to use a tokenized version of the prompt, and it needs to be in the same encoding way as it the model was pretrained

In [14]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### First to check how well the model is working I train it on a small part of the dataset. This way I have a faster way to try it with differt parameters and prompt format. And also important to not waste computational resurces

In [17]:
light_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
light_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [18]:
bs = 18
epochs = 3
lr = 2e-5

In [19]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
trainer = Trainer(model, args, train_dataset=light_train_dataset, eval_dataset=light_eval_dataset,
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.we

In [21]:
trainer.train();

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.578742,0.757000
2,No log,0.462439,0.798000
3,No log,0.458469,0.799000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


### the evaluation result seems pretty good, I can try it on the test set and see if it maintains similar results

In [22]:
eval = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

### The text should be in the same format on how it was finetuned, so I use the same function

In [23]:
eval = prepare_promt(eval)

In [24]:
eval.text[2]

'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all [SEP] keyword: none'

### And as explained previously it needs to be encoded also in the same way

In [25]:
eval_ds = Dataset.from_pandas(eval).map(tokenize_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

### Perform the prediction

In [26]:
preds_light = trainer.predict(eval_ds).predictions
preds_light

array([[ 0.11643945, -0.16335243],
       [-0.06887359,  0.03072335],
       [-0.8602782 ,  1.0674385 ],
       ...,
       [-0.80548126,  1.024606  ],
       [-0.6933291 ,  0.96658474],
       [-0.6348493 ,  0.7708333 ]], dtype=float32)

### And I need to extract the predictions from the 2D array

In [27]:
preds_light = np.argmax(preds_light, axis=-1)
preds_light

array([0, 1, 1, ..., 1, 1, 1])

In [28]:
eval['id']

0           0
1           2
2           3
3           9
4          11
        ...  
3258    10861
3259    10865
3260    10868
3261    10874
3262    10875
Name: id, Length: 3263, dtype: int64

### And now I can check if the result on the test set maintain the similar accuracy of the evaluation

In [29]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval['id'],
    'target': preds_light
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

22746

### I have a score of circa 81% that is pretty good, so now I can perform the finetuning on the full dataset

In [30]:
bs_full = 18
epochs_full = 3
lr_full = 2e-5

In [31]:
args_full = TrainingArguments('outputs', learning_rate=lr_full, warmup_ratio=0.1,
    evaluation_strategy="epoch", per_device_train_batch_size=bs_full, per_device_eval_batch_size=bs_full*2,
    num_train_epochs=epochs_full, weight_decay=0.01, report_to='none')

In [32]:
model_full = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
trainer_full = Trainer(model_full, args_full, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["test"],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.we

In [33]:
trainer_full.train();

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.412486,0.825345
2,No log,0.421822,0.826658
3,0.382400,0.415619,0.827315


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


### I can see that now I have a better accuracy, so I can perform again the prediction, this time with the new model and submit it

In [34]:
preds_full = trainer_full.predict(eval_ds).predictions
preds_full = np.argmax(preds_full, axis=-1)

submission = datasets.Dataset.from_dict({
    'id': eval['id'],
    'target': preds_full
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

22746